In [51]:
import copy
import json
import pickle
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize
from io import StringIO
import math
import time

url_all_matches_TI19 = 'https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(matches.leagueid%20%3D%2010749)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272018-08-06T22%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%2010000'
url_base_match = 'https://api.opendota.com/api/matches/'

class DotaTeam:
    def __init__(self,name,wins,losses,winrate,skill,real,luck):
        self.name = name
        self.wins = wins
        self.losses = losses
        self.winrate = winrate
        self.skill = skill
        self.real = real
        self.luck = luck
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return str(self)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__,
            sort_keys=True, indent=4)

class DotaMatch:
    def __init__(self,match_id,radiant_score,dire_score,duration,first_blood_time,picks_bans,radiant_team,dire_team):
        self.match_id = match_id
        self.radiant_score = radiant_score
        self.dire_score = dire_score
        self.duration = duration
        self.first_blood_time = first_blood_time
        self.pick_bans = picks_bans
        self.radiant_team = radiant_team
        self.dire_team = dire_team
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return str(self)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__,
            sort_keys=True, indent=4)

def save_obj(obj, name):
    with open('./Data/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('./Data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
# r = requests.get(url)
r_all_matches_TI19 = requests.get(url_all_matches_TI19)

In [8]:
parsed_json_all_matches_TI19 = json.loads(r_all_matches_TI19.text)
df_all_matches_TI19 = pd.json_normalize(parsed_json_all_matches_TI19)


list_all_matches_TI19 = []

In [9]:
for match in parsed_json_all_matches_TI19['rows']:
    if match['match_id'] not in list_all_matches_TI19:
        list_all_matches_TI19.append(match['match_id'])
print('Number of Matches: ', len(list_all_matches_TI19))

Number of Matches:  461


In [10]:
small_dataset = list_all_matches_TI19 #[:10]

In [11]:
fetch_counter = 1
matchdata_TI19 = []
for match in small_dataset:
    # r = requests.get(url)
    matchdata_response = requests.get(url_base_match + str(match))
    if matchdata_response.status_code != 200:
        raise TypeError("Did not get 200 response code")
    matchdata_TI19.append(json.loads(matchdata_response.text))
    if fetch_counter % 10 == 0:
        print('Request count: ', fetch_counter)
    fetch_counter +=1
    time.sleep(1.5)


Request count:  10
Request count:  20
Request count:  30
Request count:  40
Request count:  50
Request count:  60
Request count:  70
Request count:  80
Request count:  90
Request count:  100
Request count:  110
Request count:  120
Request count:  130
Request count:  140
Request count:  150
Request count:  160
Request count:  170
Request count:  180
Request count:  190
Request count:  200
Request count:  210
Request count:  220
Request count:  230
Request count:  240
Request count:  250
Request count:  260
Request count:  270
Request count:  280
Request count:  290
Request count:  300
Request count:  310
Request count:  320
Request count:  330
Request count:  340
Request count:  350
Request count:  360
Request count:  370
Request count:  380
Request count:  390
Request count:  400
Request count:  410
Request count:  420
Request count:  430
Request count:  440
Request count:  450
Request count:  460


In [12]:
clean_matchdata_TI19 = copy.deepcopy(matchdata_TI19)

team_list_TI19 = []
team_name_list = []

real_arr = []
skill_arr = []
luck_arr = []

# Create list of team names which are competing
for match in matchdata_TI19:
    if 'radiant_team' in match.keys():
        if match['radiant_team']['name'] not in team_name_list:
            team_name_list.append(match['radiant_team']['name'])
    else:
        clean_matchdata_TI19.remove(match)
    if 'dire_team' in match.keys():
        if match['dire_team']['name'] not in team_name_list:
            team_name_list.append(match['dire_team']['name'])
    else:
        if match in clean_matchdata_TI19:
            clean_matchdata_TI19.remove(match)

# Store matchdata as file

Length raw:  461
Length cleaned:  442
number_games_per_season:  442
number_games_per_team:  0.13122171945701358
SD real:  0.1193368316929387
SD skill:  0.1193368316929387
SD luck:  3.469446951953614e-18
Contribution luck:  2.907272551764005e-17
Contribution skill:  1.0


In [53]:
type(clean_matchdata_TI19[0])
save_obj(clean_matchdata_TI19, 'clean_matchdata_TI19')

In [ ]:
# Uncomment in case data has been lost
# clean_matchdata_TI19 = load_obj('clean_matchdata_TI19')

In [55]:
print('Length raw: ', len(matchdata_TI19))
print('Length cleaned: ', len(clean_matchdata_TI19))

# Create dota teams object from name list        
for name in team_name_list:
    team_list_TI19.append(DotaTeam(name, 0, 0, 0, 0, 0, 0))

# Increment win / loss values from results
for match in clean_matchdata_TI19:
    if match['radiant_win']:
        winner_team = match['radiant_team']['name']
        looser_team = match['dire_team']['name']
    else:
        winner_team = match['dire_team']['name']
        looser_team = match['radiant_team']['name']

    for team in team_list_TI19:
        if team.name == winner_team:
            team.wins +=1
        if team.name == looser_team:
            team.losses +=1
            
# Summary numbers
number_games_per_season = len(clean_matchdata_TI19)
number_games_per_team = len(team_list_TI19)/len(clean_matchdata_TI19)

print('number_games_per_season: ', number_games_per_season)
print('number_games_per_team: ', number_games_per_team)

            
# Calculate winrate and cointoss
for team in team_list_TI19:
    team.winrate = team.wins/(team.wins + team.losses)
    team.real = (number_games_per_season - team.wins) / (len(team_list_TI19)-1)
    team.luck = math.sqrt(0.5*(0.5/number_games_per_season))
    team.skill = team.real - team.luck
    
for team in team_list_TI19:
    real_arr.append(team.real)
    skill_arr.append(team.skill)
    luck_arr.append(team.luck)
    
sd_real = np.std(real_arr)
sd_skill = np.std(skill_arr)
sd_luck = np.std(luck_arr)

contribution_luck = sd_luck / sd_real
contribution_skill = sd_skill / sd_real

print('SD real: ', sd_real)
print('SD skill: ', sd_skill)
print('SD luck: ', sd_luck)
print('Contribution luck: ', contribution_luck)
print('Contribution skill: ', contribution_skill)


Length raw:  461
Length cleaned:  442
Length cleaned:  442


In [43]:
df = pd.DataFrame([team.__dict__ for team in team_list_TI19])
df

,name,wins,losses,winrate,skill,real,luck
0,BOOM ID,3,4,0.428571,7.677972,7.701754,0.023783
1,Mineski,21,16,0.567568,7.362182,7.385965,0.023783
2,Adroit,8,8,0.500000,7.590253,7.614035,0.023783
3,Team Jinesbrus,12,7,0.631579,7.520077,7.543860,0.023783
4,Team EVOS,2,5,0.285714,7.695516,7.719298,0.023783
5,496 Gaming,3,4,0.428571,7.677972,7.701754,0.023783
6,RESURGENCE,1,6,0.142857,7.713060,7.736842,0.023783
7,AMPLFY,6,6,0.500000,7.625340,7.649123,0.023783
8,Natus Vincere,19,17,0.527778,7.397270,7.421053,0.023783
9,Gambit Esports,3,4,0.428571,7.677972,7.701754,0.023783
